<a href="https://colab.research.google.com/github/putrik06/Question_Answering_NLP/blob/main/S1_ANLP_assign2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2


# A. Tasks as specified for your team structure

## Load dataset

In [ ]:
# Get some libraries
import pandas as pd
import numpy as np
import nltk
import regex as re
import seaborn as sns
import json
import spacy
import matplotlib.pyplot as plt

In [ ]:
# Read the dataset from google drive
from google.colab import drive
drive.mount('/content/drive')

# Path to my JSON file
archive_path ='/content/drive/My Drive/Colab Notebooks/NLP/NLP_A2/news_dataset.csv'

# Get data from JSON file
encoding_type = 'latin1'
df = pd.read_csv(archive_path, encoding=encoding_type)

# Display DataFrame information
df.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1000 non-null   int64 
 1   author   994 non-null    object
 2   date     1000 non-null   object
 3   year     1000 non-null   object
 4   month    1000 non-null   object
 5   topic    1000 non-null   object
 6   article  1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


In [ ]:
df

,id,author,date,year,month,topic,article
0,17307,Marlise Simons,1/01/2017,2017,1,architecture,PARIS ? When the Islamic State was about to...
1,17292,Andy Newman,31/12/2016,2016,12,art,Angels are everywhere in the Mu?iz family?s ap...
2,17298,Emma G. Fitzsimmons,2/01/2017,2017,1,business,Finally. The Second Avenue subway opened in Ne...
3,17311,Carl Hulse,3/01/2017,2017,1,business,WASHINGTON ? It?s or time for Republica...
4,17339,Jim Rutenberg,5/01/2017,2017,1,business,"For Megyn Kelly, the shift from Fox News to NB..."
...,...,...,...,...,...,...,...
995,18460,Gerry Mullany,14/03/2017,2017,3,accidents,HONG KONG ? Hundreds of pilot whales that s...
996,18461,Rory Smith,10/02/2017,2017,2,sports,"NICE, France ? Riv?re accepts the complim..."
997,18462,Jack Ewing,9/02/2017,2017,2,business,FRANKFURT ? Germans who never really warmed...
998,18463,Scott Cacciola,10/02/2017,2017,2,sports,Charles Oakley has strong feelings about compe...


In [ ]:
df.drop(columns=['author','date', 'year','month', 'topic'], inplace=True)

In [ ]:
df

,id,article
0,17307,PARIS ? When the Islamic State was about to...
1,17292,Angels are everywhere in the Mu?iz family?s ap...
2,17298,Finally. The Second Avenue subway opened in Ne...
3,17311,WASHINGTON ? It?s or time for Republica...
4,17339,"For Megyn Kelly, the shift from Fox News to NB..."
...,...,...
995,18460,HONG KONG ? Hundreds of pilot whales that s...
996,18461,"NICE, France ? Riv?re accepts the complim..."
997,18462,FRANKFURT ? Germans who never really warmed...
998,18463,Charles Oakley has strong feelings about compe...


In [ ]:
df['article'] = df['article'].str.replace('?', '')

In [ ]:
# df['article'] = df['article'].str.lower()

In [ ]:
# df=df.loc[ :100, :]

## Experiment

In [ ]:
import spacy
import pandas as pd

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Function to extract named entities and build an index
def build_index(df):
  entity_index = {}
  for index, row in df.iterrows():
    # Apply the nlp model to the text and store the Doc object in the dictionary
    entity_index[row['id']] = list(set(nlp(row['article'].lower()).ents))
  return entity_index

# Build the index
index = build_index(df)

# Print the index
# print(index)

In [ ]:
print(index)

{17307: [paris, islamic, march, ubelmann, 36, syria, paris, syrian, four days, four, six, ubelmann, ubelmann, one, today, syria, iraq, islamic, ubelmann, paris, jan. 9, 40, several weeks ago, france, the middle eastern, islamist, iraqi, b. c., damascus, christian, ubelmann, syrian, bashar, syrian, the last year, some 20, syria, iraq, islamic, some 3, 000, years, nimrud, november, 2015, nimrud, assyrian, syrian, paris, europe, the middle east, 150, syria, islamic, iraq, syria, yemen, saudi, pascal butterlin, paris, butterlin, more than 20 years, mari, iraq, iraq, syrian, france, syrian, first, 2012, ali, 2014, islamic, ali, paris, french, syria, france, the first half of the 20th century, hollande, this past month, france, $30 million, $100 million, hollande, france, syrian, two], 17292: [bronx, jos, zoraida muiz, puerto rico, jos, vietnam, 14, 29, new york, 1983, 1987, muiz, eight, first, jos jr., 2, six, first, muiz, 50, bronx, muiz, second, jesus, 3, 2007, muiz, muiz, muizs, jos jr.,

In [ ]:
def find_matching_articles(query, index):
    # Extract entity texts from the query and convert them to lowercase
    query_ents = set( nlp(query.lower()).ents)
    print("Query entities:", query_ents)
    matching_articles = set()

    # Iterate over each article ID and its associated entities in the index
    for id, entities in index.items():
        # Normalize the list of entities by converting them to lowercase and stripping extra quotes
        normalized_entities = set(entities)
        print(f"Processing article ID {id} with entities {normalized_entities}")

        # Check if all query entities are present in the article's entities
        if query_ents.issubset(normalized_entities):
            matching_articles.add(id)
            print(f"Match found: {id}")  # Debug: Match found
        else:
            print(f"No match found: {id}")  # Debug: No match found

    print("Finished iterating over all items.")  # Debug: Ensure loop completes
    return matching_articles


In [ ]:
def build_inverted_index(df):
  inverted_index = {}  # This will store the entity as the key and a list of document IDs as the value

  for index, row in df.iterrows():
    doc_id = row['id']
    doc = nlp(row['article'].lower())  # Process the article text with nlp
    for ent in doc.ents:
      ent_text = ent.text.lower().strip()  # Normalize the entity text
      if ent_text not in inverted_index:
        inverted_index[ent_text] = [doc_id]
      else:
        inverted_index[ent_text].append(doc_id)

    # Remove potential duplicates in document ID lists
    for ent, doc_ids in inverted_index.items():
      inverted_index[ent] = list(set(doc_ids))

    return inverted_index

# Call the function to build the inverted index
inverted_index = build_inverted_index(df)



paris: [18433, 17541, 18040, 18192, 18320, 18450, 17428, 18454, 17307, 17308, 18333, 17950, 17823, 18461, 17441, 18462, 17956, 18086, 17703, 18342, 17321, 18351, 18358, 17975, 17592, 18231, 18236, 17726, 17347, 17732, 17861, 18243, 18374, 17865, 18121, 17613, 17741, 18127, 17360, 18253, 17746, 18258, 18134, 18266, 18395, 18268, 17891, 18404, 17385, 17902, 18417, 18034, 18163, 17525, 17909, 17399, 18038, 17401, 18043, 18301, 17919]


In [ ]:
# Print out the inverted index for the entity 'paris'
if 'samsung' in inverted_index:
    print('samsung:', inverted_index['samsung'])
else:
    print('No documents found for paris')

samsung: [17312, 17764, 17734, 17574, 18375, 17487, 17657, 17851]


In [ ]:
# Example query
query = "where is Paris"
matched_articles = find_matching_articles(query, index)

print("Matched articles:", matched_articles)

Query entities: {paris}
Processing article ID 17307 with entities {ubelmann, paris, syria, more than 20 years, iraq, 40, islamic, first, ubelmann, two, syria, syria, syrian, syrian, syrian, paris, islamic, iraqi, saudi, ali, french, the last year, iraq, pascal butterlin, nimrud, jan. 9, france, $30 million, syrian, four, paris, syrian, hollande, the middle eastern, the first half of the 20th century, 2012, nimrud, assyrian, 2014, christian, syrian, 36, some 3, 000, years, yemen, syria, france, november, march, ubelmann, bashar, islamic, syria, 2015, paris, iraq, islamist, butterlin, paris, iraq, ubelmann, islamic, paris, some 20, france, $100 million, islamic, france, ali, hollande, four days, ubelmann, this past month, the middle east, damascus, iraq, today, europe, six, b. c., 150, several weeks ago, mari, syrian, france, syria, one}
No match found: 17307
Processing article ID 17292 with entities {august, new york, jos jr., december, puerto rico, jos, jos jr., jos jr., muiz, jesus, o

In [ ]:
# !pip install spacy

### Information Retrieval

#### BM25

In [ ]:
query= 'Who is the vice chairman of Samsung?'

In [ ]:
# pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi

In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tokenized_query = word_tokenize(query)
tokenized_query

['Who', 'is', 'the', 'vice', 'chairman', 'of', 'Samsung', '?']

In [ ]:
df['article_tokenized']= df['article'].apply(lambda x:word_tokenize(x))


In [ ]:
corpus=df

In [ ]:
bm25 = BM25Okapi(df['article_tokenized'])
doc_scores = bm25.get_scores(tokenized_query)

In [ ]:
top_5_doc=bm25.get_top_n(tokenized_query, df['article_tokenized'], n=5) #ranked in order of relevance, with the most relevant document first

In [ ]:
for doc in top_5_doc:
  print(" ".join(doc))

SEOUL , South Korea ? A special prosecutor investigating the corruption scandal that led to President Park ? s impeachment summoned the de facto head of Samsung for questioning on Wednesday , calling him a bribery suspect . The de facto leader , Jay Y. Lee , the vice chairman of Samsung , will be questioned on Thursday , according to the special prosecutor ? s office , which recommended that he also be investigated on suspicion of perjury . Mr. Lee effectively runs Samsung , South Korea ? s largest conglomerate he is the son of its chairman , Lee who has been incapacitated with health problems . He is expected to be asked whether donations that Samsung made to two foundations controlled by Choi a longtime friend of the president , amounted to bribes , and what role , if any , he played in the decision to give the money . Investigators at the special prosecutor ? s office have questioned other senior Samsung executives as suspects about the bribery accusations . Neither Samsung nor Mr. 

#### BERT

In [ ]:
# pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to encode pairs of query and document using BERT
def encode_pairs(tokenized_query, tokenized_documents, tokenizer, model):
  query = " ".join(tokenized_query)
  cls_embeddings = []
  for tokenized_doc in tokenized_documents:
      document = " ".join(tokenized_doc)
      # Tokenize and encode the sequences as a pair
      inputs = tokenizer.encode_plus(query, document, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
      outputs = model(**inputs)
      # Retrieve the [CLS] token's embeddings
      cls_embedding = outputs.last_hidden_state[:, 0, :]
      cls_embeddings.append(cls_embedding)
  return cls_embeddings



# Encode each query-document pair
pair_embeddings = encode_pairs(tokenized_query, top_5_doc, tokenizer, model)

# Flatten the embeddings tensor to 1D for scoring
pair_scores = [torch.flatten(embed).detach().numpy() for embed in pair_embeddings]

# Rank the documents based on the scores
# The score here could be the sum of the embeddings or any other metric you determine to be relevant
sorted_doc_indices = sorted(range(len(pair_scores)), key=lambda i: pair_scores[i].sum())

# Sort the documents according to the scores
sorted_docs = [top_5_doc[i] for i in sorted_doc_indices]

# Print out the sorted documents
print("Sorted documents based on BERT [CLS] embeddings:")
for doc in sorted_docs:
    print(f"Document: {' '.join(doc)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens 

Sorted documents based on BERT [CLS] embeddings:
Document: SEOUL , South Korea ? A special prosecutor investigating the corruption scandal that led to President Park ? s impeachment summoned the de facto head of Samsung for questioning on Wednesday , calling him a bribery suspect . The de facto leader , Jay Y. Lee , the vice chairman of Samsung , will be questioned on Thursday , according to the special prosecutor ? s office , which recommended that he also be investigated on suspicion of perjury . Mr. Lee effectively runs Samsung , South Korea ? s largest conglomerate he is the son of its chairman , Lee who has been incapacitated with health problems . He is expected to be asked whether donations that Samsung made to two foundations controlled by Choi a longtime friend of the president , amounted to bribes , and what role , if any , he played in the decision to give the money . Investigators at the special prosecutor ? s office have questioned other senior Samsung executives as suspec

### Answer Span Extraction

In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/PY3/english.pickle')

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
import nltk
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.util import ngrams

In [ ]:
# Function to create n-grams from word tokens
def create_ngrams(word_tokens, n=10):
  # Generate n-grams
  ngram_list = list(ngrams(word_tokens, n))

  # Convert n-grams to string format
  ngram_strings = [' '.join(ngram) for ngram in ngram_list]

  return ngram_strings

In [ ]:
# Define a function which returns the cosine similarity between 2 vectors
def cosine(query, document):
  return np.dot(query,document) / (np.linalg.norm(query) * np.linalg.norm(document))

In [ ]:

# List to store tuples of (sentence, similarity)
similarities = []

# Loop through each tokenized sentence in the 'split_sentence' column
for sent_list in article['10_grams']:
    #Encode each tokenized sentence using SBERT model
  for tokenized_sent in sent_list:
    sent_embedding = sbert_model.encode([tokenized_sent])[0]

    # Calculate similarity between query vector and sentence embedding
    sim = cosine(query_vec, sent_embedding)

    # Append the tuple of (sentence, similarity) to the list
    similarities.append((tokenized_sent, sim))

# Sort the list based on similarities in descending order
similarities.sort(key=lambda x: x[1], reverse=True)

# Get the top 5 entries
top_5_sentences = similarities[:5]

# Print the top 5 sentences with their cosine similarities
for i, (sent, sim) in enumerate(top_5_sentences, start=1):
  print(f"Top {i} Sentence = {sent}; similarity = {sim}")

## Test

### Information retrieval

### Answer Span Extraction

## Proposal

In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
from nltk.util import ngrams

def cosine_similarity(query, document):
  return np.dot(query, document) / (np.linalg.norm(query) * np.linalg.norm(document))

def create_ngrams(word_tokens, n=10):
  if len(word_tokens) < n:
    return []  # Return empty list if not enough tokens for one n-gram
  ngram_list = list(ngrams(word_tokens, n))
  ngram_strings = [' '.join(ngram) for ngram in ngram_list]
  return ngram_strings

def question_answer(query):

  tokenized_query = word_tokenize(query)
  corpus=df['article'].apply(lambda x:word_tokenize(x))

  # Information Retrieval with BM25
  bm25 = BM25Okapi(corpus)
  doc_scores = bm25.get_scores(tokenized_query)
  top_doc=bm25.get_top_n(tokenized_query, corpus, n=1)
  first=" ". join(top_doc[0])

  # Answer Sentence Extraction using Sentence Embeddings
  sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')
  query_vec = sbert_model.encode([' '.join(tokenized_query)])[0]
  article_10_grams = create_ngrams(word_tokenize(first), n=10)

  similarities = []
  for gram in article_10_grams:
    gram_vec = sbert_model.encode([gram])[0]
    sim = cosine_similarity(query_vec, gram_vec)
    similarities.append((gram, sim))

    # Sort and return the highest similarity n-gram
    if not similarities:
      return None  # Handle case where no n-grams are generated
    similarities.sort(key=lambda x: x[1], reverse=True)
  return similarities[0]


In [ ]:
question_answer('Who is the vice chairman of Samsung')

(', Jay Y. Lee , the vice chairman of Samsung', 0.9505648)

In [ ]:
question_answer('How many charges of coercing on Ms. Choi')

(', state prosecutors indicted Ms. Choi on charges of coercing', 0.79896456)

In [ ]:
while True:
  user_question = input("Enter your question (or type 'quit' to exit): ")

  if user_question.lower() == 'quit':
      break

  # Get and print the answer
  print("Answer:", question_answer(user_question))


# B. References

# C. Appendix